In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
import scipy
import glob
import sklearn 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import optimizers
from keras.layers import Input, Activation, Dense, LeakyReLU
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint

In [2]:
# Data import
wt_filtered =glob.glob('wt_filtered*50.lccdata')

# filtered wt LCC data import
wt_f_var_names = ['wt_3f', 'wt_12f', 'wt_20f']
    
for var, file in zip(wt_f_var_names, wt_filtered):
    globals()[var] = pd.read_csv(file,sep='\t').drop(columns = 'Unnamed: 0')

# filtered mutant LCC data import

D132H_filtered =glob.glob('D132H_filtered*_50.lccdata')

D132H_f_var_names = ['D132H_3f', 'D132H_12f', 'D132H_20f']
    
for var, file in zip(D132H_f_var_names, D132H_filtered):
    globals()[var] = pd.read_csv(file, sep='\t').drop(columns = 'Unnamed: 0')

In [3]:
# Visualization of dataset

print('WT for window size = 3')
display(wt_3f)

print('\n')
print('---------------------------------')
print('D132H for window size = 20')
display(D132H_20f)

WT for window size = 3


,16,19,29,30
0,17.3049,30.7508,24.7567,23.1634
1,22.4792,28.1640,23.8363,23.2798
2,25.5636,30.2936,24.8655,23.4604
3,26.4733,29.5746,21.3842,22.6630
4,24.0147,27.5496,23.8249,22.3185
...,...,...,...,...
39995,14.7258,8.4287,24.3534,22.6400
39996,15.5602,7.1757,24.9417,24.1508
39997,15.4532,7.1490,23.8513,27.4276
39998,15.1188,7.5725,25.4096,26.8701




---------------------------------
D132H for window size = 20


,7,20,26,27
0,18.7095,46.4135,38.2248,42.8092
1,13.4667,46.4414,34.7611,41.1095
2,15.7000,47.2786,30.2701,34.9940
3,15.4481,44.8032,33.4548,38.2540
4,20.1022,44.0575,33.0935,39.0292
...,...,...,...,...
39995,6.9673,28.1672,15.1968,12.9232
39996,7.2460,27.9588,15.2995,12.7038
39997,7.3270,27.8777,16.1102,13.1354
39998,7.2212,28.3756,16.5540,13.4088


In [5]:
# Concateneate wt and mutant dataframes and rename columns

wt_f = pd.concat([wt_3f, wt_12f, wt_20f], axis = 1)
    
D132H_f = pd.concat([D132H_3f, D132H_12f, D132H_20f], axis = 1)

colnames = [*range(0,10)]
colnames
wt_f.columns = colnames
D132H_f.columns = colnames


In [6]:
# Visualization of dataset
print('Filtered wt data')
display(wt_f)

print('\n')
print('---------------------------------')
print('Filtered D132H data')
display(D132H_f)

Filtered wt data


,0,1,2,3,4,5,6,7,8,9
0,17.3049,30.7508,24.7567,23.1634,14.2374,40.1886,6.3249,9.6718,6.0082,9.6574
1,22.4792,28.1640,23.8363,23.2798,14.5602,36.3229,6.3105,10.6216,5.9996,11.2509
2,25.5636,30.2936,24.8655,23.4604,17.4329,41.2223,5.7233,11.4243,7.3021,11.5329
3,26.4733,29.5746,21.3842,22.6630,20.1207,39.2809,5.4864,10.7079,6.4156,10.7262
4,24.0147,27.5496,23.8249,22.3185,15.6335,38.7009,5.4673,12.0023,6.3573,11.2367
...,...,...,...,...,...,...,...,...,...,...
39995,14.7258,8.4287,24.3534,22.6400,16.9755,31.9599,4.9712,8.9677,10.1179,8.9430
39996,15.5602,7.1757,24.9417,24.1508,17.0192,29.9824,4.8021,9.0623,10.1026,8.6752
39997,15.4532,7.1490,23.8513,27.4276,18.5272,29.4126,5.9590,10.3438,8.6309,9.4556
39998,15.1188,7.5725,25.4096,26.8701,17.1766,32.6983,4.3996,10.3760,9.5901,9.3513




---------------------------------
Filtered D132H data


,0,1,2,3,4,5,6,7,8,9
0,32.2144,41.6730,36.3674,31.2519,13.4055,41.6856,18.7095,46.4135,38.2248,42.8092
1,32.2545,39.3565,35.5082,29.8643,12.0150,40.2262,13.4667,46.4414,34.7611,41.1095
2,30.2448,42.7255,33.4780,30.7351,17.0081,41.7220,15.7000,47.2786,30.2701,34.9940
3,31.1229,42.3124,35.9424,31.0427,15.6520,40.4408,15.4481,44.8032,33.4548,38.2540
4,32.7239,40.8462,34.5238,27.3446,20.7450,38.0932,20.1022,44.0575,33.0935,39.0292
...,...,...,...,...,...,...,...,...,...,...
39995,23.9089,22.5600,12.8804,14.6848,14.0324,23.2605,6.9673,28.1672,15.1968,12.9232
39996,22.1314,22.9405,13.7161,15.5070,14.9455,22.4160,7.2460,27.9588,15.2995,12.7038
39997,20.4286,22.6088,13.4875,15.1243,14.0516,22.7481,7.3270,27.8777,16.1102,13.1354
39998,24.2292,22.1429,13.4961,15.4669,14.0350,23.5958,7.2212,28.3756,16.5540,13.4088


In [7]:
# Data pre processing

def preprocessing(wt, mutant):
    
    wt_label = np.zeros(len(wt)) # Set wt labels to 0
    
    mutant_label = np.ones(len(mutant))
    
    # Concatenate data frames and label arrays

    X_train_full = pd.concat([wt.reset_index(), mutant.reset_index()])
    y_train_full = np.concatenate((wt_label, mutant_label))
    
    #Drop index column and normalise training data
    X_train_full = X_train_full.drop(columns = 'index')
    
    X_train_full= X_train_full.div(100)
    
    # Separate training and validation sets and print relevant shapes
    
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, stratify=y_train_full, test_size=0.2)
    
    print(X_train.shape)
    print(X_valid.shape)
    print(y_train.shape)
    print(y_valid.shape)
    
    return X_train, X_valid, y_train, y_valid

In [8]:
X_train_f, X_valid_f, y_train_f, y_valid_f = preprocessing(wt_f, D132H_f)

(64000, 10)
(16000, 10)
(64000,)
(16000,)


In [ ]:
# Save shuffled validation datasets (for use in downstream analysis)
X_valid_f.to_csv('X_val.csv')

y_valid_f_df = pd.DataFrame({'class':y_valid_f})
y_valid_f_df.to_csv('y_val_df.csv')

In [10]:
# Get autoencoder model
def get_ae(train_data, LeReLU_alpha=0.01):
    
    #Input layer
    input_layer = Input(shape=(train_data.shape[1]), name='ae_input')
    
    encoder = Dense(256, activation=LeakyReLU(alpha=LeReLU_alpha), name='e1')(input_layer)
    encoder = Dense(64, activation=LeakyReLU(alpha=LeReLU_alpha), name='e2')(encoder)

    encoded = Dense(2, activation=LeakyReLU(alpha=LeReLU_alpha), name='ae_latent')(encoder)
    
    decoder = Dense(64, activation=LeakyReLU(alpha=LeReLU_alpha), name='d1')(encoded)
    decoder = Dense(256, activation=LeakyReLU(alpha=LeReLU_alpha), name='d2')(decoder)

    output_layer = Dense(train_data.shape[1], activation=LeakyReLU(alpha=LeReLU_alpha), name='ae_output')(decoder)
    
    model = Model(input_layer, output_layer)

    return model

In [27]:
# Get ae for filtered data
autoencoder = get_ae(X_train_f)

In [28]:
# Print summary of ae model
autoencoder.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ae_input (InputLayer)       [(None, 10)]              0         
                                                                 
 e1 (Dense)                  (None, 256)               2816      
                                                                 
 e2 (Dense)                  (None, 64)                16448     
                                                                 
 ae_latent (Dense)           (None, 2)                 130       
                                                                 
 d1 (Dense)                  (None, 64)                192       
                                                                 
 d2 (Dense)                  (None, 256)               16640     
                                                                 
 ae_output (Dense)           (None, 10)                2570

In [29]:
# Compile the model
autoencoder.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(learning_rate = 0.005))

In [30]:
# Train ae and extract latent layer after each epoch set

for counts in tqdm(range(100)): #this determines the number of epoch sets
    history = autoencoder.fit(X_train_f, X_train_f, batch_size = 256, 
                              epochs = 1000, validation_data = (X_valid_f, X_valid_f), verbose=1)
    name = "02_FILT_SAE_complex_0.2_lr5_10e5_bs256_e1000_ws3_12_20f"
    # convert history object to dataframe and plot rates
    training_history = pd.DataFrame(history.history)
    plt.plot (training_history);
    file_name_0 = name + "_training_history" + str(counts)
    training_history.to_pickle(file_name_0)
    file_name_1 = name + str(counts) + "_#1.png"
    plt.savefig(file_name_1, dpi=300)
    plt.clf()
    
    # read in latent layer
    dr_model = tf.keras.models.Model(inputs  = autoencoder.get_layer('ae_input').input, 
                                     outputs = autoencoder.get_layer('ae_latent').output)
    dr_model.summary()
    
    # put the validation data through current latent layer model
    x = []
    y = []
    z = []
    for i in range(16000):
        z.append(y_valid_f[i])
        op = dr_model.predict(np.array([X_valid_f.iloc[i]]))
        x.append(op[0][0])
        y.append(op[0][1])

    df = pd.DataFrame()
    df['x'] = x
    df['y'] = y
    df['z'] = ["trajectory-" + str(k) for k in z]
 
    plt.figure(figsize = (8, 6));
    fig = sns.scatterplot(x = 'x', y='y', hue='z', data=df, s=10)
    file_name_2 = name + str(counts) + "_#2.png"
    fig.figure.savefig(file_name_2, dpi = 300)
    plt.clf()
    
    #
    file_name_3 = '02_ws3_12_20_f'+str(counts)
    df.to_pickle(file_name_3)

    file_name = 'models/saved_model_FILT_02_' + str(counts)
    autoencoder.save(file_name) 
    

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.0025 - val_loss: 9.7282e-04
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 8.3932e-04 - val_loss: 7.6654e-04
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 7.5513e-04 - val_loss: 7.1825e-04
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 7.1937e-04 - val_loss: 7.0641e-04
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 7.0069e-04 - val_loss: 6.9315e-04
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 6.8876e-04 - val_loss: 6.8673e-04
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 6.7891e-04 - val_loss: 6.6477e-04
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 6.6990e-04 - val_loss: 6.7533e-04
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 6.6182e-04 - val_loss: 6.7563e-04
Epoch 10/100
250/250 [=

INFO:tensorflow:Assets written to: models/saved_model_FILT_02_0\assets


INFO:tensorflow:Assets written to: models/saved_model_FILT_02_0\assets
  1%|▊                                                                             | 1/100 [10:21<17:05:37, 621.59s/it]

Epoch 1/100
250/250 [==============================] - 1s 4ms/step - loss: 5.4977e-04 - val_loss: 5.6087e-04
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 5.6218e-04 - val_loss: 5.4907e-04
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 5.5748e-04 - val_loss: 5.6512e-04
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 5.6581e-04 - val_loss: 5.5283e-04
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 5.5604e-04 - val_loss: 5.4013e-04
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 5.5512e-04 - val_loss: 5.4346e-04
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 5.5084e-04 - val_loss: 5.7533e-04
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 5.5544e-04 - val_loss: 5.4752e-04
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 5.5080e-04 - val_loss: 5.6676e-04
Epoch 10/100
250/25

...

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

In [63]:
# check
#print(X_train_f.index.to_list() ==  y_train_f.index.to_list())
#print(X_valid_f.index.to_list() ==  y_valid_f.index.to_list())

# Save index list
shuffled_index_train = X_train_f.index.to_list()
shuffled_index_valid = X_valid_f.index.to_list()

# open a binary file in write mode
file = open("shufftrain", "wb")
# save array to the file
np.save(file, shuffled_index_train)
# close the file
file.close
# open a binary file in write mode
file = open("shuffval", "wb")
# save array to the file
np.save(file, shuffled_index_valid)
# close the file
file.close

<function BufferedWriter.close>